**Eric Meinhardt / emeinhardt@ucsd.edu**

In [1]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import os
import csv

In [3]:
from collections import Counter

In [4]:
from random import choice

In [5]:
from functools import reduce
from itertools import takewhile, product

In [6]:
# from joblib import Parallel, delayed

# J = 10
# BACKEND = 'multiprocessing'
# # BACKEND = 'loky'
# V = 10
# PREFER = 'processes'
# # PREFER = 'threads'

# def par(gen_expr):
#     return Parallel(n_jobs=J, backend=BACKEND, verbose=V, prefer=PREFER)(gen_expr)

# def identity(x):
#     return x

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview-&amp;-Dependencies" data-toc-modified-id="Overview-&amp;-Dependencies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview &amp; Dependencies</a></span><ul class="toc-item"><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Dependencies</a></span></li><li><span><a href="#Outputs" data-toc-modified-id="Outputs-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Outputs</a></span></li></ul></li><li><span><a href="#Reading-in-the-data" data-toc-modified-id="Reading-in-the-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Reading in the data</a></span><ul class="toc-item"><li><span><a href="#CMU-pronouncing-dictionary" data-toc-modified-id="CMU-pronouncing-dictionary-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>CMU pronouncing dictionary</a></span></li></ul></li><li><span><a href="#Applying-the-flapping-rule-/-creating-surface-forms" data-toc-modified-id="Applying-the-flapping-rule-/-creating-surface-forms-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Applying the flapping rule / creating surface forms</a></span><ul class="toc-item"><li><span><a href="#Functions-for-manipulating-transcriptions" data-toc-modified-id="Functions-for-manipulating-transcriptions-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Functions for manipulating transcriptions</a></span></li><li><span><a href="#Carving-up-the-inventory-and-defining-symbol-classes" data-toc-modified-id="Carving-up-the-inventory-and-defining-symbol-classes-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Carving up the inventory and defining symbol classes</a></span></li><li><span><a href="#Applying-the-flapping-rule" data-toc-modified-id="Applying-the-flapping-rule-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Applying the flapping rule</a></span></li><li><span><a href="#Creating-a-list-of-positive-input-output-examples" data-toc-modified-id="Creating-a-list-of-positive-input-output-examples-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Creating a list of positive input-output examples</a></span></li></ul></li></ul></div>

# Overview & Dependencies

The goal of this notebook is processing the CMU pronouncing dictionary to create (input, output) / (underlying, surface) pairs for learning the American English flapping rule discussed in Gildea & Jurafsky's 1996 paper "Learning bias and phonological-rule induction".

## Dependencies

 - **Transcriptions:** The version of the CMU pronouncing dictionary processed here (and assumed to be in the working directory) is taken from https://github.com/emeinhardt/cmu-ipa. Please see the documentation there for more on what processing goes into that file.
 - **`Unix`-like OS:** Some Unix-like shell commands are used throughout, though they aren't essential.

## Outputs

Currently the only output of the notebook is a `.tsv` file with positive examples of flappable URs and the flapped SRs.

# Reading in the data

## CMU pronouncing dictionary

In [7]:
os.getcwd()
root_dir = os.getcwd()

'/mnt/cube/home/AD/emeinhar/gj-flap-data'

In [8]:
os.listdir()

['Recreating the Gildea & Jurafsky (1996) Flapping Data.ipynb',
 '__pycache__',
 '.ipynb_checkpoints',
 '.gitignore',
 'COCA',
 '.git',
 'cmudict-0.7b_IPA_stressed.tsv',
 'positive_flapping_io_examples.tsv']

In [9]:
%cat -n cmudict-0.7b_IPA_stressed.tsv | head -60





























































cat: write error: Broken pipe


In [10]:
cmudict_filename = 'cmudict-0.7b_IPA_stressed.tsv'

In [11]:
lexicon_relation = []
with open(cmudict_filename, 'r',) as csvfile:
        reader = csv.DictReader(csvfile, delimiter='\t', quoting=csv.QUOTE_NONE)
        for row in reader:
            lexicon_relation.append(row)

In [12]:
lexicon_relation[:5]

[OrderedDict([('Orthography', '!EXCLAMATION-POINT'),
              ('Transcription', 'ɛ2.k.s.k.l.ʌ0.m.eɪ1.ʃ.ʌ0.n.p.ɔɪ2.n.t')]),
 OrderedDict([('Orthography', '"CLOSE-QUOTE'),
              ('Transcription', 'k.l.oʊ1.z.k.w.oʊ1.t')]),
 OrderedDict([('Orthography', '"DOUBLE-QUOTE'),
              ('Transcription', 'd.ʌ1.b.ʌ0.l.k.w.oʊ1.t')]),
 OrderedDict([('Orthography', '"END-OF-QUOTE'),
              ('Transcription', 'ɛ1.n.d.ʌ0.v.k.w.oʊ1.t')]),
 OrderedDict([('Orthography', '"END-QUOTE'),
              ('Transcription', 'ɛ1.n.d.k.w.oʊ1.t')])]

In [13]:
lexicon = lexicon_relation

# Applying the flapping rule / creating surface forms

The flapping rule (in the ARPABET and approximately representing stress as Gildea & Jurafsky do) is $$\text{t} \rightarrow \text{dx} / V' r^* \_\_ V$$.

Below I write code for enumerating each of the dotted strings (i.e. substrings in the CMU transcriptions) that matches the condition for applying the rule.

## Functions for manipulating transcriptions

In [14]:
ds2t = lambda ds: tuple(ds.split('.'))
t2ds = lambda ts: '.'.join(ts)

In [15]:
union = lambda Ss: reduce(set.union, Ss)

In [16]:
word_to_inv = lambda ds: set(ds2t(ds))
lex_to_inv = lambda Ws: union(map(word_to_inv,
                                   Ws))
get_transcription = lambda r: r['Transcription']
get_transcriptions = lambda L: list(map(get_transcription,
                                        L))

In [17]:
transcriptions = get_transcriptions(lexicon)

Below is some code I have lying around for other purposes - it manipulates k-factors of a string (represented as a dotted string or as a tuple).

In [18]:
def dsToKfactors(k, ds):
    seq = ds2t(ds)
    l = len(seq)
    if k > l:
        return tuple()
    kFactor_start_indices = takewhile(lambda pair: pair[0] <= l-k, enumerate(seq))
    kFactors = tuple(seq[index[0]:index[0]+k] for index in kFactor_start_indices)
    return set(map(t2ds, kFactors))

def dsTo2factors(ds):
    return dsToKfactors(2, ds)

def dsTo3factors(ds):
    return dsToKfactors(3, ds)

def lexiconToKfactors(DSs, k):
    myDsToKfactors = lambda ds: dsToKfactors(k, ds)
    return union(map(set, map(myDsToKfactors, DSs)))

def lexiconTo2factors(DSs):
    return union(map(set, map(dsTo2factors, DSs)))

def lexiconTo3factors(DSs):
    return union(map(set, map(dsTo3factors, DSs)))

def compareKfactors(DSs_A, DSs_B, k):
    A = lexiconToKfactors(DSs_A, k)
    B = lexiconToKfactors(DSs_B, k)
    return {"A == B":A == B, "A - B": A - B, "B - A": B - A}

def sameKfactors(DSs_A, DSs_B, k):
    return compareKfactors(DSs_A, DSs_B, k)["A == B"]

def hasIllicitKfactors(W, illicit_k_factors):
    if type(W) == str:      
        # gather the k-factors into an immutable data structure
        illicit_kfs = tuple(illicit_k_factors)
        # get the set of k-factor lengths (values of k) among the illicit_kfs
        illicit_factor_lengths = set([len(ds2t(kf)) for kf in illicit_kfs])
        # map each k to the set of k-factors of dotted string ds
        kFactorSets = {kf_l:dsToKfactors(kf_l, W) for kf_l in illicit_factor_lengths}
        illegal_kfactors_discovered = tuple(ikf for ikf in illicit_kfs if ikf in kFactorSets[len(ds2t(ikf))])
        if illegal_kfactors_discovered == tuple():
            return False
        return illegal_kfactors_discovered
    else:
        myFunc = lambda w: hasIllicitKfactors(w, illicit_k_factors)
        results = tuple(map(myFunc, W))
        if not any(results):
            return False
        return set(t2ds(each) for each in results if each != False)

def dsToKfactorSequence(k, ds):
    seq = ds2t(ds)
    l = len(seq)
    if k > l:
        return tuple()
    kFactor_start_indices = takewhile(lambda pair: pair[0] <= l-k, enumerate(seq))
    kFactors = tuple(seq[index[0]:index[0]+k] for index in kFactor_start_indices)
    return tuple(map(t2ds, kFactors))

def threeFactorSequenceToDS(threeFactors):
    wLE = ds2t(threeFactors[0])[0]
    wRE = ds2t(threeFactors[-1])[-1]
    w_NE = '.'.join([ds2t(eachTriphone)[1] for eachTriphone in threeFactors])
    return '.'.join([wLE, w_NE, wRE])

## Carving up the inventory and defining symbol classes

In [19]:
inventory = lex_to_inv(transcriptions)
len(inventory)
inventory

69

{'aɪ0',
 'aɪ1',
 'aɪ2',
 'aʊ0',
 'aʊ1',
 'aʊ2',
 'b',
 'd',
 'dʒ',
 'eɪ0',
 'eɪ1',
 'eɪ2',
 'f',
 'g',
 'h',
 'i0',
 'i1',
 'i2',
 'j',
 'k',
 'l',
 'm',
 'n',
 'oʊ0',
 'oʊ1',
 'oʊ2',
 'p',
 's',
 't',
 'tʃ',
 'u0',
 'u1',
 'u2',
 'v',
 'w',
 'z',
 'æ0',
 'æ1',
 'æ2',
 'ð',
 'ŋ',
 'ɑ0',
 'ɑ1',
 'ɑ2',
 'ɔ0',
 'ɔ1',
 'ɔ2',
 'ɔɪ0',
 'ɔɪ1',
 'ɔɪ2',
 'ɚ0',
 'ɚ1',
 'ɚ2',
 'ɛ0',
 'ɛ1',
 'ɛ2',
 'ɪ0',
 'ɪ1',
 'ɪ2',
 'ɹ',
 'ʃ',
 'ʊ0',
 'ʊ1',
 'ʊ2',
 'ʌ0',
 'ʌ1',
 'ʌ2',
 'ʒ',
 'θ'}

In [20]:
vowels = {s for s in inventory if s[-1] in {'0', '1', '2'}}
len(vowels)
vowels

45

{'aɪ0',
 'aɪ1',
 'aɪ2',
 'aʊ0',
 'aʊ1',
 'aʊ2',
 'eɪ0',
 'eɪ1',
 'eɪ2',
 'i0',
 'i1',
 'i2',
 'oʊ0',
 'oʊ1',
 'oʊ2',
 'u0',
 'u1',
 'u2',
 'æ0',
 'æ1',
 'æ2',
 'ɑ0',
 'ɑ1',
 'ɑ2',
 'ɔ0',
 'ɔ1',
 'ɔ2',
 'ɔɪ0',
 'ɔɪ1',
 'ɔɪ2',
 'ɚ0',
 'ɚ1',
 'ɚ2',
 'ɛ0',
 'ɛ1',
 'ɛ2',
 'ɪ0',
 'ɪ1',
 'ɪ2',
 'ʊ0',
 'ʊ1',
 'ʊ2',
 'ʌ0',
 'ʌ1',
 'ʌ2'}

In [21]:
stressed_vowels = {v for v in vowels if v[-1] == '1' or v[-1] == '2'}
len(stressed_vowels)
stressed_vowels

30

{'aɪ1',
 'aɪ2',
 'aʊ1',
 'aʊ2',
 'eɪ1',
 'eɪ2',
 'i1',
 'i2',
 'oʊ1',
 'oʊ2',
 'u1',
 'u2',
 'æ1',
 'æ2',
 'ɑ1',
 'ɑ2',
 'ɔ1',
 'ɔ2',
 'ɔɪ1',
 'ɔɪ2',
 'ɚ1',
 'ɚ2',
 'ɛ1',
 'ɛ2',
 'ɪ1',
 'ɪ2',
 'ʊ1',
 'ʊ2',
 'ʌ1',
 'ʌ2'}

How many consecutive `ɹ`s are there actually in the transcriptions?

In [22]:
R = 'ɹ'
hasR = lambda ds: R in ds2t(ds)
hasRR = lambda ds: t2ds((R,R)) in ds2t(ds)
hasRRR = lambda ds: t2ds((R,R,R)) in ds2t(ds)

In [23]:
R_words = {1:list(filter(hasR, transcriptions)),
           2:list(filter(hasRR, transcriptions)),
           3:list(filter(hasRRR, transcriptions))}

In [24]:
len(R_words[1])
len(R_words[2])
len(R_words[3])

43343

0

0

At most 1.

In [25]:
zero_or_one_Rs = {'', R}

In [26]:
isNotEpsilon = lambda s: s != ''
removeEpsilon = lambda ts: tuple(filter(isNotEpsilon, 
                                        ts))
removeEpsilon(('i','','t','u'))

('i', 't', 'u')

Ok. Now we can enumerate all the LHSs for rules applying to strings of (stress-annotated) IPA symbols that together constitute a desugaring of the flapping rule...

In [27]:
LHSs = tuple(map(lambda ts: t2ds(removeEpsilon(ts)),
                 product(stressed_vowels, zero_or_one_Rs, 't', vowels)))
len(LHSs)
LHSs[:10]

2700

('ɑ1.t.ɑ1',
 'ɑ1.t.æ1',
 'ɑ1.t.ɑ2',
 'ɑ1.t.ɚ0',
 'ɑ1.t.eɪ2',
 'ɑ1.t.aʊ0',
 'ɑ1.t.eɪ1',
 'ɑ1.t.i1',
 'ɑ1.t.ɔ1',
 'ɑ1.t.oʊ1')

## Applying the flapping rule

In [28]:
flap = 'ɾ'
flap_substring_map = {lhs:lhs.replace('t', flap)
                      for lhs in LHSs}

def flappable(ds):
    return ds in LHSs

def has_flappable_substrings(ds):
    threeFactorSeq = dsToKfactorSequence(3, ds)
    return any(map(flappable,
                   threeFactorSeq))

def flappable_substrings(ds):
    if not has_flappable_substrings(ds):
        return tuple()
    threeFactorSeq = dsToKfactorSequence(3, ds)
    flappable3factors = tuple(filter(flappable,
                                     threeFactorSeq))
    return flappable3factors

def flap(ds_in):
    threeFactorSeq = dsToKfactorSequence(3, ds_in)
    print(threeFactorSeq)
    flapped_3FS = tuple([flap_substring_map.get(f, f)
                         for f in threeFactorSeq])
    print(flapped_3FS)
    ds_out = threeFactorSequenceToDS(flapped_3FS)
    return ds_out

In [29]:
flappable_wordforms = {w for w in transcriptions if has_flappable_substrings(w)}
# flappable_wordforms = set(par(delayed(identity)(w) for w in transcriptions if has_flappable_substrings(w)))
len(flappable_wordforms)

6457

In [30]:
wordforms_with_multiple_flapping_opportunities = {w for w in flappable_wordforms if len(flappable_substrings(w)) > 1}
# wordforms_with_multiple_flapping_opportunities = set(par(delayed(identity)(w) 
#                                                          for w in flappable_wordforms 
#                                                          if len(flappable_substrings(w)) > 1))
len(wordforms_with_multiple_flapping_opportunities)
wordforms_with_multiple_flapping_opportunities

56

{'d.ɛ1.t.ʌ0.n.eɪ2.t.ɚ0',
 'd.ɛ1.t.ʌ0.n.eɪ2.t.ɚ0.z',
 'd.ɛ1.t.ʌ0.n.eɪ2.t.ɪ0.ŋ',
 'd.ɛ1.t.ʌ0.n.eɪ2.t.ʌ0.d',
 'k.æ2.t.ʌ0.l.ɪ1.t.ɪ0.k',
 'k.ɑ2.t.ɑ0.l.ɑ0.n.oʊ1.t.oʊ0',
 'k.ʌ0.m.p.j.u1.t.ɚ0.dʒ.ɛ1.n.ɚ0.eɪ2.t.ʌ0.d',
 'k.ʌ0.n.k.æ1.t.ʌ0.n.eɪ2.t.ɪ0.ŋ',
 'k.ʌ0.n.k.æ1.t.ʌ0.n.eɪ2.t.ʌ0.d',
 'l.ɪ1.t.ɪ0.g.eɪ2.t.ɪ0.d',
 'l.ɪ1.t.ɪ0.g.eɪ2.t.ɪ0.ŋ',
 'l.ɪ1.t.ʌ0.g.eɪ2.t.ɚ0',
 'l.ɪ1.t.ʌ0.g.eɪ2.t.ɚ0.z',
 'l.ɪ2.t.ɚ0.ɑ1.t.i0',
 'm.aɪ2.t.ɔ1.t.ɪ0.k',
 'm.i1.t.i2.t.ɪ0.ŋ',
 'm.j.u1.t.eɪ2.t.ɪ0.d',
 'm.j.u1.t.ʌ0.l.eɪ2.t.ɪ0.d',
 'm.j.u1.t.ʌ0.l.eɪ2.t.ɪ0.ŋ',
 'm.j.u1.t.ʌ0.l.eɪ2.t.ʌ0.d',
 'm.oʊ1.t.ʌ0.v.eɪ2.t.ɪ0.ŋ',
 'm.oʊ1.t.ʌ0.v.eɪ2.t.ʌ0.d',
 'm.ɪ1.t.ʌ0.g.eɪ2.t.ɪ0.d',
 'm.ɪ1.t.ʌ0.g.eɪ2.t.ɪ0.ŋ',
 'n.ɑ2.n.ɔ2.t.oʊ0.m.oʊ1.t.ɪ0.v',
 's.ɛ2.m.aɪ0.ɔ2.t.ʌ0.m.æ1.t.ɪ0.k',
 's.ɛ2.m.i0.ɔ2.t.ʌ0.m.æ1.t.ɪ0.k',
 's.ɛ2.m.ɪ0.ɔ2.t.ʌ0.m.æ1.t.ɪ0.k',
 't.i2.t.oʊ2.t.eɪ1.l.ɚ0',
 't.ɪ1.t.ʌ0.l.eɪ2.t.ɪ0.d',
 't.ɪ1.t.ʌ0.l.eɪ2.t.ɪ0.ŋ',
 'tʃ.ɪ1.t.ɚ0.tʃ.æ1.t.ɚ0',
 'w.ɔ1.t.ɚ0.b.ɔ2.t.ʌ0.l',
 'w.ɔ1.t.ɚ0.b.ɔ2.t.ʌ0.l.z',
 'ɔ1.t.ʌ0.m.eɪ2.t.ɪ0.d',
 'ɔ1.t.ʌ0.m

In [31]:
flappable_example_wordform = choice(list(flappable_wordforms))
flappable_example_wordform
flap(flappable_example_wordform)

's.æ1.t.ɚ0.f.i1.l.d'

('s.æ1.t', 'æ1.t.ɚ0', 't.ɚ0.f', 'ɚ0.f.i1', 'f.i1.l', 'i1.l.d')
('s.æ1.t', 'æ1.ɾ.ɚ0', 't.ɚ0.f', 'ɚ0.f.i1', 'f.i1.l', 'i1.l.d')


's.æ1.ɾ.ɚ0.f.i1.l.d'

In [32]:
flappable_example_wordform2 = choice(list(wordforms_with_multiple_flapping_opportunities))
flappable_example_wordform2
flap(flappable_example_wordform2)

'tʃ.ɪ1.t.ɚ0.tʃ.æ1.t.ɚ0'

('tʃ.ɪ1.t', 'ɪ1.t.ɚ0', 't.ɚ0.tʃ', 'ɚ0.tʃ.æ1', 'tʃ.æ1.t', 'æ1.t.ɚ0')
('tʃ.ɪ1.t', 'ɪ1.ɾ.ɚ0', 't.ɚ0.tʃ', 'ɚ0.tʃ.æ1', 'tʃ.æ1.t', 'æ1.ɾ.ɚ0')


'tʃ.ɪ1.ɾ.ɚ0.tʃ.æ1.ɾ.ɚ0'

In [33]:
flap('ɔ1.l.t.ɚ0.n.eɪ2.t.ɚ0')

('ɔ1.l.t', 'l.t.ɚ0', 't.ɚ0.n', 'ɚ0.n.eɪ2', 'n.eɪ2.t', 'eɪ2.t.ɚ0')
('ɔ1.l.t', 'l.t.ɚ0', 't.ɚ0.n', 'ɚ0.n.eɪ2', 'n.eɪ2.t', 'eɪ2.ɾ.ɚ0')


'ɔ1.l.t.ɚ0.n.eɪ2.ɾ.ɚ0'

In [ ]:
tfs = ('ɔ1.l.t', 'l.t.ɚ0', 't.ɚ0.n', 'ɚ0.n.eɪ2', 'n.eɪ2.t', 'eɪ2.ɾ.ɚ0')
tfs[0]
tfs[0][0]
tfs[-1]
tfs[-1][-1]

'ɔ1.l.t'

'ɔ'

'eɪ2.ɾ.ɚ0'

'0'

In [ ]:
threeFactorSequenceToDS(tfs)

'ɔ1.l.t.ɚ0.n.eɪ2.ɾ.ɚ0'

## Creating a list of positive input-output examples

In [ ]:
positive_io_examples = [{'UR':w_in,
                         'SR':flap(w_in)}
                        for w_in in flappable_wordforms]
len(positive_io_examples)
list(positive_io_examples)[:10]

('l.aɪ1.t', 'aɪ1.t.ɪ0', 't.ɪ0.n', 'ɪ0.n.ʌ0', 'n.ʌ0.n')
('l.aɪ1.t', 'aɪ1.ɾ.ɪ0', 't.ɪ0.n', 'ɪ0.n.ʌ0', 'n.ʌ0.n')
('k.oʊ1.t', 'oʊ1.t.ʌ0', 't.ʌ0.l', 'ʌ0.l.ɚ0')
('k.oʊ1.t', 'oʊ1.ɾ.ʌ0', 't.ʌ0.l', 'ʌ0.l.ɚ0')
('w.ɪ1.t', 'ɪ1.t.i0')
('w.ɪ1.t', 'ɪ1.ɾ.i0')
('ɹ.oʊ0.s', 'oʊ0.s.ɛ1', 's.ɛ1.t', 'ɛ1.t.oʊ0')
('ɹ.oʊ0.s', 'oʊ0.s.ɛ1', 's.ɛ1.t', 'ɛ1.ɾ.oʊ0')
('k.ʌ0.m', 'ʌ0.m.b', 'm.b.æ1', 'b.æ1.t', 'æ1.t.ɪ0', 't.ɪ0.v')
('k.ʌ0.m', 'ʌ0.m.b', 'm.b.æ1', 'b.æ1.t', 'æ1.ɾ.ɪ0', 't.ɪ0.v')
('s.i0.æ1', 'i0.æ1.t', 'æ1.t.ʌ0', 't.ʌ0.l', 'ʌ0.l.z')
('s.i0.æ1', 'i0.æ1.t', 'æ1.ɾ.ʌ0', 't.ʌ0.l', 'ʌ0.l.z')
('l.ɛ1.t', 'ɛ1.t.ɚ0', 't.ɚ0.m', 'ɚ0.m.ʌ0', 'm.ʌ0.n', 'ʌ0.n.z')
('l.ɛ1.t', 'ɛ1.ɾ.ɚ0', 't.ɚ0.m', 'ɚ0.m.ʌ0', 'm.ʌ0.n', 'ʌ0.n.z')
('ɹ.oʊ1.t', 'oʊ1.t.ʌ0', 't.ʌ0.n')
('ɹ.oʊ1.t', 'oʊ1.ɾ.ʌ0', 't.ʌ0.n')
('ɹ.oʊ0.s', 'oʊ0.s.ɛ1', 's.ɛ1.t', 'ɛ1.t.i0')
('ɹ.oʊ0.s', 'oʊ0.s.ɛ1', 's.ɛ1.t', 'ɛ1.ɾ.i0')
('m.j.u1', 'j.u1.t', 'u1.t.ʌ0', 't.ʌ0.l', 'ʌ0.l.eɪ2', 'l.eɪ2.t')
('m.j.u1', 'j.u1.t', 'u1.ɾ.ʌ0', 't.ʌ0.l', 'ʌ0.l.eɪ2', 'l.eɪ2.t')
('p.ɛ1.t', 'ɛ1.t.i

limit_output extension: Maximum message size of 20000 exceeded with 101086 characters

In [ ]:
positive_io_example_fn = 'positive_flapping_io_examples.tsv'

In [ ]:
with open(positive_io_example_fn, 'w') as file:
    writer = csv.DictWriter(file, ['UR','SR'], delimiter='\t')
    writer.writeheader()
    for ex in positive_io_examples:
        writer.writerow(ex)

37

33

21

35

39

...

21

29

27

39

39

In [ ]:
%cat -n /mnt/cube/home/AD/emeinhar/gj-flap-data/positive_flapping_io_examples.tsv | head -60